# Data and Model Parallelism in PyTorch

## The Supervised Training Loop

Before diving into data and model parallelism, let's remind ourselves the how basic supervised training loop in PyTorch.  Each of those steps have to be done in the right order, no matter what kind of parallelism we are using; how they're done (and what kind of computational or communications resources are used) is what changes.

So as in the figure below, we'll start with a dataset, which we load into a `DataLoader` to get batches of data.  For each batch, we:

1. **Load the data**: Get a batch of data from the `DataLoader`.
2. **Forward pass**: Pass the data through the model to get predictions.
3. **Compute loss**: Compare the predictions to the ground truth labels to compute the loss.
4. **Backward pass**: Compute the gradients of the loss with respect to the model parameters.
5. **Update parameters**: Use an optimizer to update the model parameters based on the gradients.

Pretty straightforward.

![Single GPU training data flow](images/single-gpu-training.png)

## Different approaches to parallelism

### More, faster, or larger?

If you have multiple GPUs available, you can use them to speed up the training jobs you have to do.  You can use them in parallel; the question is, you'll be parallelizing over _what_; what will you be distributing over the multiple GPUs?

Parallel computing broadly can be thought of as having (at least) one of three goals:
1. **More** jobs - with more computational resourecs, you can do more similar jobs in the same amount of time
2. **Faster** job - with more computational resources, you can do one job in less time
3. **Larger** job - with more computational resources, you can do one job that is larger than you could do on a single machine.

The 'more' case here is the simplest one, and we won't really cover it in this workshop.  A simple example might be a hyperparameter sweep, where you run the same training job with different hyperparameters on each GPU, looking for an optimal set.  This is a very simple and effective way to use multiple GPUs, but it doesn't require any of the parallelism techniques we'll be covering here.

### Faster - Data Parallelism

Data parallelism is principally about speeding up the training of a single model by increasing the training rate (samples/second).  The idea is to have multiple replicas (say $P$) of the same model, each one being fed a separate $1/P$ of the data, thus notionally completing each epoch $P$ times faster.   Data parallism is shown on the left in the figure below.

This doesn't come without cost. Each replica of the model has to be kept in sync with the others, so that they all have the same parameters.  This is done by aggregating the gradients layer-by-layer from each replica before having the optimizer updating the parameters.  Besides the communications overhead, there's then a little bit of extra computation as all replicas have to perform the optimization step and update the optimizer state.

![Data and model parallelism approaches differ in how they take advantage of multiple GPUs](images/parallelisms.png)

### Larger - Model Parallelism

Model parallelism is about training larger models than can fit on a single GPU.  The idea is to split the model into $P$ parts, each part being placed on a different GPU.  Each GPU then only has to store and compute the parameters for its part of the model, and the model can be larger than any single GPU's memory.  Model parallelism is shown on the right in the figure above.

You can think of the two kinds of model parallelism as being **horizontal** and **vertical**, describing how the model is split.  Pipeline parallelism makes slices vertically through the model, spliting the model by layers, with different units of layers each being placed on a different GPU.  ALthernately Tensor parallelism slices vertically through each layer, cutting the tensors into shards.

These have different communication overheads compared to data parellism.  In pipeline parallelism, comunication is relatively stratighforward each GPU only has to communicate with the next one forward or backward in the pipeline for the forward and backward steps; it is completley avilable for the parameters and gradients for layers in its unit.

In tensor parallelism, each GPU has to gather much or all of the other activations from other shards to do the forward and backward passes. 

## These Paralelism Techniques are Orthogonal, and Can Compose

We're mostly going to be considering these techniques separately in this workshop, but it's worth noting that they actually combine together very nicely, because the dimensions along which they are being parallelized are orthogonal.

You can absolutely combine (say) data and pipeline parallelism by implementing a pipeline-parallelized model over 4 GPUs (for example), and then running two replicas of that to speed up the training by ensuring each replica only has to see half of the data in an epoch.   Same with tensor parallelism and data parallelism, or any other combination of these techniques.

In fact, [FSDP](https://docs.pytorch.org/tutorials/intermediate/FSDP_tutorial.html) (Fully Sharded Data Parallel) which we'll touch on today, implements many of these approaches very efficiently.  You can think of the "units" in FSDP as being the pipeline parallelism units, and the shards of the tensors as being the tensor parallelism.  The data parallelism is implemented by having multiple replicas of the model, each one being fed a separate $1/P$ of the data.

![FSDP combines tensor and pipeline parallelism; from the FSDP paper](images/fsdp.png)

## The Communication / Computation Tradeoff

The reason we talk about model parallelism being principally about training larger models is that there is significant communication overhead.  Even though using more GPUs gives you access to more computing power, using model-parallelism for a small model that fits comfortably on a single GPU will almost certainly result in a very inefficient training job --- maybe slower than on a single GPU! --- because of the communication overhead.

The table below gives very rough estimates of the communication overhead for communications to different parts of the computing system, compared to a simple computation over the same amount of time.  "Nearby" communication - between GPUs on the same node - isn't too bad, but when you have to communicate between nodes, or offload data to the CPU or storage, data transfer can be very slow indeed.

A corollary of this is that computation is extremely fast, and some of the approaches that (say) FSDP enable are to recompute data rather that moving it back and forth.

| Operation | Estimated Time for 100 kB | Time (microseconds) | Relative to moving data to/from global memory |
| --- | --- | --- | --- |
| Simple scalar computation	 |  | 0.005	| 0.01 |
| Move data to global memory on the same GPU	|  0.75 µs	| 0.75	| 1.00 |
| Move data to another GPU on the same NVLINK domain | 2.5 µs	| 2.5 | 3.50 |
| Move data to CPU memory over PCIe	| 10 µs	| 10 |	13.0 |
| Move data to another GPU on another node over Infiniband	| 1 ms	| 1000	| 1,300 |
| Move data to a local SSD filesystem | 30 ms	| 33500	| 45,000 |
| Move data to a networked filesystem |	80 ms	| 85000	| 110,000 |

## Shrinking The Memory Footprint of the Model

One of the reasons for model parallelism is to be able to train larger models than can fit on a single GPU; you'd like the model to fit on as few GPUs as possible.   So model parallelism appraoches are often discussed in the same combination as techniques to reduce the memory footprint of the model.

We'll look at some of those techniques in the FSDP section, but it's worth noting that they can be used with (or without!) any of the parallelism techniques we've discussed so far.  Techniques like:

* Model quantization (reduce the size/precision of the floating point numbers used in different stage of the model)
* Gradient checkpointing 
* Activitation offloading to CPU when workign on other layers
* Recomputation of activations instead of storing them

all come up frequently, and will be touched on in the FSDP section.


# Summary

We've touched on some of the basic concepts of data and model parallelism in PyTorch, and how they can be used to speed up training jobs or train larger models.  We'll be looking at how to implement these techniques in practice in the next sections, starting with data parallelism.

See you [at the next notebook!](4_Disibuted_data_parallel.ipynb)